In [1]:

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob
import h5py
import time

In [7]:
def ls2sol(ls):
    N_s = 668.6
    ls_peri = 250.99
    t_peri = 485.35
    a = 1.52368
    e = 0.09340
    epsilon = 25.1919
    if (ls == 0).any():
        ls = .01
    nu = np.radians(ls) + 1.90258
    E = np.arctan((np.tan(nu/2))/(np.sqrt((1+e)/(1-e))))*2
    M = E - e*np.sin(E)
    Ds = (M/(2*np.pi))*N_s + t_peri
    if (Ds < 0).any():
        Ds = Ds + N_s
    if (Ds > N_s).any():
        Ds = Ds - N_s
    return(Ds)

In [10]:
sg = xr.open_dataset('StupidGridFull.nc', group='flux')
files = [f for f in glob.glob("redsun_timeseries_*.nc")]

j3pvF = np.zeros((19,37))
j3pvi_bg1_F = np.zeros((19,37))
j3pvi_bg2_F = np.zeros((19,37))
j3pvi_bg3_F = np.zeros((19,37))

In [17]:
tic = time.perf_counter()
for file in files[0:1]:
    ds = xr.open_dataset(file)
    lat = ds['lat'][0]
    lon = ds['lon'][0]
    G = np.zeros(len(ds['lon']))
    for ri in range(0,len(ds['lon'])):
        ls = ds['ls'][ri]
        hr = ds['hr'][ri]
        G[ri] = sg['flux_dw_sw'][lat,lon,ls,hr]
    lss = np.unique(ds['ls'])

    try:
        P = G[:, np.newaxis, np.newaxis, np.newaxis] * ds['j3_etaPV_3bg']
        zz = np.zeros((len(lss),len(ds['j3-bg1']),len(ds['j3-bg2']),len(ds['j3-bg3'])))
        for i in range(0,len(lss)):
            hr_int = np.where(ds['ls']==lss[i])
            x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * 60 * 60)
            for j in range(0,len(ds['j3-bg1'])):
                for k in range(0,len(ds['j3-bg2'])):
                    for l in range(0,len(ds['j3-bg3'])):
                        y = P[:,j,k,l][hr_int]
                        z = np.trapz(y,x=x)
                        zz[i,j,k,l] = z
        z = np.zeros((len(ds['j3-bg1']),len(ds['j3-bg2']),len(ds['j3-bg3'])))
        x = ls2sol(lss) * 1.02569 * 24 * 60 * 60
        for j in range(0,len(ds['j3-bg1'])):
            for k in range(0,len(ds['j3-bg2'])):
                for l in range(0,len(ds['j3-bg3'])):
                    y = zz[:,j,k,l]
                    z[j,k,l] = np.trapz(y,x=lss)
        j3pv = np.max(z)
        j3pvi = np.unravel_index(np.argmax(z),np.shape(z), order='C')

        j3pvF[lat,lon] = j3pv * (1/688) * (1/1000) * (1/3600)
        j3pvi_bg1_F[lat,lon] = j3pvi[0]
        j3pvi_bg2_F[lat,lon] = j3pvi[1]
        j3pvi_bg3_F[lat,lon] = j3pvi[2]
    except:
        print(file)

lats = np.arange(-90,91,10)
lons = np.arange(-180,181,10)
test = xr.Dataset(
    {'j3pv': (('lon','lat'), j3pvF.transpose()),
    'j3pv-bg1': (('lon','lat'), j3pvi_bg1_F.transpose()),
    'j3pv-bg2': (('lon','lat'), j3pvi_bg2_F.transpose()),
    'j3pv-bg3': (('lon','lat'), j3pvi_bg3_F.transpose())},
    coords = {
        'lon': lons,
        'lat': lats,
    },)

test['lat'].attrs['units'] = 'degrees_north'
test['lon'].attrs['units'] = 'degrees_east'
test['j3pv'].attrs['units'] = '(kW*hr)/(day*m^2)'
test['j3pv-bg1'].attrs['units'] = 'nm'
test['j3pv-bg2'].attrs['units'] = 'nm'
test['j3pv-bg3'].attrs['units'] = 'nm'

test['j3pv-bg1'] = test['j3pv-bg1'].where(test['j3pv-bg1'] != 0.0)
test['j3pv-bg2'] = test['j3pv-bg2'].where(test['j3pv-bg2'] != 0.0)
test['j3pv-bg3'] = test['j3pv-bg3'].where(test['j3pv-bg3'] != 0.0)


test.to_netcdf('j3pv.nc')
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

Downloaded the tutorial in 899.6079 seconds
